## PyTorch Resnets

In [11]:
import pandas as pd
import numpy as np
import os
import matplotlib.pyplot as plt
import json
from copy import deepcopy
import random
import sys
import time
from typing import List
from pprint import PrettyPrinter

pp = PrettyPrinter(indent=4)

# get an absolute path to the directory that contains parent files
project_dir = globals()["_dh"][0]
sys.path.append(os.path.normpath(os.path.join(project_dir, "..", "..", "..", "..")))

from experiments.utils.constants import (
    PIPLINES_PATH,
    NODE_PROFILING_CONFIGS_PATH,
    NODE_PROFILING_RESULTS_PATH,
)
from experiments.utils.parser import Parser

In [13]:
series = 500
experiment_id = 1
config_key_mapper = "key_config_mapper.csv"
model_name = "resnet-human"
series_path = os.path.join(NODE_PROFILING_RESULTS_PATH, "series", str(series))
loader = Parser(
    series_path=series_path, config_key_mapper=config_key_mapper, model_name=model_name
)
results = loader.result_processing()
key_config_df = loader.key_config_mapper()
# print(results.columns)
# print(key_config_df.columns)
# results

In [19]:
# results = loader._read_results(selected)
# final_dataframe = []
# for experiment_id, result in results.items():
#     processed_exp = {'experiment_id': int(experiment_id)}
#     flattened_results = self.flatten_results(
#         results[str(experiment_id)]['responses'])
#     if log is not None:
#         latencies, timeout_count = self.latency_calculator(
#             flattened_results, log[experiment_id])
#     else:
#         latencies, timeout_count = self.latency_calculator(
#             flattened_results, log)

In [14]:
results.columns

Index(['experiment_id', 'client_to_model_avg',
       'client_to_model_p99', 'client_to_model_p50',
       'client_to_model_var', 'client_to_model_max',
       'client_to_model_min', 'model_latencies_avg',
       'model_latencies_p99', 'model_latencies_p50', 'model_latencies_var',
       'model_latencies_max', 'model_latencies_min',
       'model_to_client_avg', 'model_to_client_p99',
       'model_to_client_p50', 'model_to_client_var',
       'model_to_client_max', 'model_to_client_min',
       'start_time', 'end_time', 'duration', 'timeout_count',
       'cpu_usage_count_avg', 'cpu_usage_count_p99', 'cpu_usage_count_p50',
       'cpu_usage_count_var', 'cpu_usage_count_max', 'cpu_usage_count_min',
       'cpu_usage_rate_avg', 'cpu_usage_rate_p99', 'cpu_usage_rate_p50',
       'cpu_usage_rate_var', 'cpu_usage_rate_max', 'cpu_usage_rate_min',
       'cpu_throttled_count_avg', 'cpu_throttled_count_p99',
       'cpu_throttled_count_p50', 'cpu_throttled_count_var',
       'cpu_throttled_co

In [15]:
configs = loader.load_configs()
for config_name, config in configs.items():
    print(f"File name: {config_name}")
    pp.pprint(config)

File name: 0.yaml
{   'benchmark_duration': 1,
    'cpu_request': ['2', '4'],
    'data_type': 'image',
    'max_batch_size': ['1'],
    'max_batch_time': ['1'],
    'memory_request': ['10Gi'],
    'mode': 'exponential',
    'model_variants': ['resnet18', 'resnet34'],
    'node_name': 'resnet-human',
    'num_interop_threads': ['1'],
    'num_threads': ['1'],
    'pipeline_name': 'video',
    'repetition': 1,
    'replicas': [1],
    'series': 500,
    'series_meta': 'all variables change for making the profiler 1,\\n cpu '
                   'type: Intel(R) Xeon(R) Gold 6126 CPU @ 2.60GHz',
    'timeout': 1,
    'use_threading': 'True',
    'workload_config': {'load_duration': 30, 'loads_to_test': [1]},
    'workload_type': 'static'}


In [16]:
display(key_config_df)

,experiment_id,pipeline_name,node_name,model_variant,cpu_request,memory_request,max_batch_size,max_batch_time,load,load_duration,series,series_meta,replicas,no_engine,mode,data_type,benchmark_duration
0,1,video,resnet-human,resnet18,2,10Gi,1,1,1,30,500,all variables change for making the profiler 1...,1,False,exponential,image,1
1,2,video,resnet-human,resnet18,4,10Gi,1,1,1,30,500,all variables change for making the profiler 1...,1,False,exponential,image,1
2,3,video,resnet-human,resnet34,2,10Gi,1,1,1,30,500,all variables change for making the profiler 1...,1,False,exponential,image,1
3,4,video,resnet-human,resnet34,4,10Gi,1,1,1,30,500,all variables change for making the profiler 1...,1,False,exponential,image,1


In [20]:
experiment_ids = key_config_df["experiment_id"].tolist()
metadata_columns = ["model_variant", "max_batch_size", "cpu_request", "load"]
results_columns = [
    "model_latencies_min",
    "model_latencies_p99",
    "cpu_usage_count_avg",
    "model_latencies_avg",
    "throughput_max",
]
output = loader.table_maker(
    experiment_ids=experiment_ids,
    metadata_columns=metadata_columns,
    results_columns=results_columns,
)
output["expected_throughput"] = (
    1 / output["model_latencies_avg"] * output["max_batch_size"]
)
# output.sort_values(by='throughput')
display(output)

,model_variant,max_batch_size,cpu_request,load,model_latencies_min,model_latencies_p99,cpu_usage_count_avg,model_latencies_avg,throughput_max,expected_throughput
0,resnet18,1,2,1,0.033338,0.086855,4.467573,0.052890,1.034483,18.907328
1,resnet18,1,4,1,0.022350,0.052250,6.619705,0.038017,1.000000,26.304344
2,resnet34,1,2,1,0.051939,0.112621,4.868188,0.074657,1.000000,13.394599
3,resnet34,1,4,1,0.044907,0.102972,3.744582,0.056413,0.966667,17.726256


In [21]:
results

,experiment_id,client_to_model_avg,client_to_model_p99,client_to_model_p50,client_to_model_var,client_to_model_max,client_to_model_min,model_latencies_avg,model_latencies_p99,model_latencies_p50,...,memory_usage_p50,memory_usage_var,memory_usage_max,memory_usage_min,throughput_avg,throughput_p99,throughput_p50,throughput_var,throughput_max,throughput_min
0,4,0.006327,0.009301,0.006105,7.742261e-07,0.009443,0.005095,0.056413,0.102972,0.052010,...,3076096.0,1.976895e+16,288763904.0,3076096.0,0.644108,0.966667,0.703651,0.090519,0.966667,0.091387
1,1,0.008238,0.035993,0.006235,5.222493e-05,0.037392,0.005136,0.052890,0.086855,0.052332,...,241500160.0,1.511522e+10,242008064.0,241500160.0,0.630712,1.021034,0.701464,0.096071,1.034483,0.068167
2,3,0.009063,0.051566,0.005957,1.140201e-04,0.057075,0.005204,0.074657,0.112621,0.074483,...,295747584.0,0.000000e+00,295747584.0,295747584.0,0.645212,1.000000,0.689677,0.097736,1.000000,0.077387
3,2,0.006307,0.013298,0.005898,3.377530e-06,0.014033,0.004605,0.038017,0.052250,0.037629,...,236269568.0,0.000000e+00,236269568.0,236269568.0,0.638961,1.000000,0.699129,0.091595,1.000000,0.086226
